In [1]:
import ee 
from ee_jupyter.ipyleaflet import Map
import ipyleaflet as ipl
import json

ee.Initialize()
with open(r'../shp/china.json','r') as shp:
    china_json = json.load(shp)

china=ee.FeatureCollection(china_json)
boundary = ee.Image().paint(china, 0, 1)
map0=Map(center=[35,110],zoom=4,basemap=ipl.basemaps.Esri.WorldStreetMap)
map0.addLayer(boundary,{'palette':'grey'},'China')
map0

Map(center=[35, 110], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

上面是一些简单的初始化, 下面我们来观察一下降水影像集是否能用

In [2]:
gpm = ee.ImageCollection("NASA/GPM_L3/IMERG_V06")
bands=gpm.first().bandNames().getInfo()
for band in bands:
    print(band)
gpm_201901 = gpm.filterDate('2019-01-01','2019-01-31').select('precipitationCal')
print(gpm_201901.size().getInfo())

HQobservationTime
HQprecipSource
HQprecipitation
IRkalmanFilterWeight
IRprecipitation
precipitationCal
precipitationUncal
probabilityLiquidPrecipitation
randomError
1440


从上面可以看到这个基本信息.

或者我们可以把查看基本信息设置为一个函数

以及我们可以把设置地图变为一个函数

嗯,好主意!

In [3]:
import sys
sys.path.append(r'../')
import my_ee_imports as ei
ei.print_bands(gpm.first())
ei.print_size(gpm_201901)

HQobservationTime
HQprecipSource
HQprecipitation
IRkalmanFilterWeight
IRprecipitation
precipitationCal
precipitationUncal
probabilityLiquidPrecipitation
randomError
1440


OK, Let's get a map of precipitation to have a look at the result

In [6]:
from ipywidgets import Layout

precipitation = gpm_201901.sum().clip(china)
visParams = {'min':0,'max':250,'palette':['red','yellow','green','blue','purple']}
map1=Map(center=[35,110],zoom=4,basemap=ipl.basemaps.Esri.WorldStreetMap)
map1.addLayer(precipitation,visParams,'precipitation')
map1.addLayer(boundary,{'palette':'grey'},'China')
map1.layout = Layout(width='100%', height='600px')
map1

Map(center=[35, 110], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

Kind of unsure about min and max value of the image.

Let's take a trial of the following codes.

In [8]:
# 函数，用于计算每个省份的最大/最小降水值
def calc_max_min_precip(feature):
    stats = precipitation.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=feature.geometry(),
        scale=1000,  # 根据需要调整
        maxPixels=1e9
    )
    return feature.set(stats)

# 应用函数到每个省份
stats_per_province = china.map(calc_max_min_precip)

# 提取并计算全国的最大/最小降水值
max_values = []
min_values = []

for i,province in enumerate(stats_per_province.getInfo()['features']):
    print(f'Processing {i+1}/{len(stats_per_province.getInfo()["features"])}')
    properties = province['properties']
    max_values.append(properties['precipitationCal_max'])
    min_values.append(properties['precipitationCal_min'])

nationwide_max = max(max_values)
nationwide_min = min(min_values)

print('Nationwide max precipitation:', nationwide_max)
print('Nationwide min precipitation:', nationwide_min)

ProxyError: HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Max retries exceeded with url: /v1/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json (Caused by ProxyError('Unable to connect to proxy', RemoteDisconnected('Remote end closed connection without response')))

Seems to have some troubles...

Let's see another ImageCollection:

In [12]:
chirps=ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
chirps_201901 = chirps.filterDate('2019-01-01','2019-01-31').select('precipitation')
ei.print_bands(chirps_201901.first())
ei.print_size(chirps_201901)

precipitation
30


In [17]:
visParams2 = {'min':0,'max':100,'palette':['red','yellow','green','blue','purple']}
map2=Map(center=[35,110],zoom=4,basemap=ipl.basemaps.OpenTopoMap)
map2.addLayer(chirps_201901.sum().clip(china),visParams2,'precipitation')
map2.addLayer(boundary,{'palette':'grey'},'China')
map2.layout = Layout(width='100%', height='600px')
map2

Map(center=[35, 110], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…